# Proyek Analisis Data: Nama dataset
- Nama: M. Fatahillah Aqsa L.B.N.
- Email: fatahillaqsa@gmail.com
- Id Dicoding: mfaqsa

## Menentukan Pertanyaan Bisnis

Dari Dataset yang telah diberikan, dapat dirumuskan pertanyaan terkait jumlah penggunaan rental sepeda sebagai berikut
1. Bagaimana korelasi antara musim dan cuaca pada tingkat penggunaan rental sepeda?
2. Perbandingan antara weekday dan holiday pada jumlah penggunaan
3. Waktu dalam 1 hari yang paling banyak tingkat penggunaannya

## Menyaipkan semua library yang dibuthkan

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import streamlit as st
from babel.numbers import format_currency
sns.set(style='dark')

## Data Wrangling

### Gathering Data

In [ ]:
day_df = pd.read_csv("day.csv")

hour_df = pd.read_csv("hour.csv")

### Assessing Data

In [ ]:
day_df['temp'] = day_df['temp']*41
day_df['atemp'] = day_df['atemp']*50
day_df['hum'] = day_df['hum']*100
day_df['windspeed'] = day_df['windspeed']*67

hour_df['temp'] = hour_df['temp']*41
hour_df['atemp'] = hour_df['atemp']*50
hour_df['hum'] = hour_df['hum']*100
hour_df['windspeed'] = hour_df['windspeed']*67

### Cleaning Data

In [ ]:
day_df.dteday = pd.to_datetime(day_df.dteday)
hour_df.dteday = pd.to_datetime(hour_df.dteday)

monthly_df = day_df.resample(rule='M', on='dteday').agg({
    "casual": "sum",
    "registered": "sum",
    "cnt": "sum"
})
monthly_df.index = monthly_df.index.strftime('%Y-%b')
monthly_df = monthly_df.reset_index()

 ## Exploratory Data Analysis (EDA)

### Explore

In [ ]:
season_df = day_df.groupby(by="season").cnt.sum().reset_index()
season_df['season'] = season_df['season'].replace({
    1:'Spring',
    2:'Summer',
    3:'Fall',
    4:'Winter'
    })

weathersit_df = day_df.groupby(by="weathersit").cnt.sum().reset_index()
weathersit_df['weathersit'] = weathersit_df['weathersit'].replace({
    1:'Clear, Partly cloudy',
    2:'Mist, Cloudy',
    3:'Light Snow, Light Rain',
    4:'Heavy rain, Ice pallets, Thunderstorm, Snow, Fog'
    })

weekday_df = day_df.groupby(by="weekday").cnt.sum().reset_index()
weekday_df['weekday'] = weekday_df['weekday'].replace({
    0:'Minggu',
    1:'Senin',
    2:'Selasa',
    3:'Rabu',
    4:'Kamis',
    5:"Jum'at",
    6:'Sabtu'})

workingday_df = day_df.groupby(by="workingday").cnt.sum().reset_index()
workingday_df['workingday'] = workingday_df['workingday'].replace({
    0:'Hari libur',
    1:'Hari kerja',
    })

hour_df['hour_group'] = hour_df.hr.apply(lambda x: 'Pagi' if x>=6 and x<12 else 'Siang' if x>=12 and x<18 else 'Sore' if x>=18 and x<21 else 'Malam')
bytime_df = hour_df.groupby(by='hour_group').cnt.sum().reset_index()
timesort_dict = {'Pagi':0 , 'Siang':1 , 'Sore':2 , 'Malam':3}
bytime_df = bytime_df.sort_values(by='hour_group', key=lambda x : x.map(timesort_dict))

## Visualization & Explanatory Analysis

In [ ]:
st.header('Bike Sharing Dataset - Capital Bikeshare System :sparkles:')

st.subheader('Penggunaan rental sepeda rentang tahun 2011-2012')

total_orders = monthly_df.cnt.sum()
st.metric("Total penggunaan", value=total_orders)

fig, ax = plt.subplots(figsize=(20, 10))
ax.plot(
    monthly_df["dteday"],
    monthly_df["cnt"],
    marker='o',
    linewidth=2,
    color="#90CAF9"
)
ax.set_title("Jumlah penggunaan rental sepeda setiap bulan", fontsize=40)
ax.tick_params(axis='y', labelsize=25)
ax.tick_params(axis='x', labelsize=20, rotation= 45)
st.pyplot(fig)

### Pertanyaan 1:

In [ ]:
st.subheader("Seasons & Weathers")

fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(40, 20))

colors1 = [ "#D3D3D3", "#D3D3D3", "#72BCD4", "#D3D3D3"]
sns.barplot(
    y="cnt",
    x="season",
    data=season_df,
    palette=colors1,
    ax=ax[0]
)

ax[0].set_title("Penggunaan rental sepeda tiap musim", loc="center", fontsize=35)
ax[0].set_ylabel(None)
ax[0].set_xlabel(None)
ax[0].tick_params(axis='y', labelsize=25)
ax[0].tick_params(axis='x', labelsize=20)

colors2 = ["#72BCD4", "#D3D3D3", "#D3D3D3", "#D3D3D3", "#D3D3D3"]
sns.barplot(
    y="cnt",
    x="weathersit",
    data=weathersit_df,
    palette=colors2,
    ax=ax[1]
)

ax[1].set_title("Penggunaan rental sepeda berdasarkan cuaca", loc="center", fontsize=35)
ax[1].set_ylabel(None)
ax[1].set_xlabel(None)
ax[1].tick_params(axis='y', labelsize=25)
ax[1].tick_params(axis='x', labelsize=20)
st.pyplot(fig)

### Pertanyaan 2:

In [ ]:
st.subheader("Berdasarkan Hari")

fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(40, 20))

colors3 = [ "#D3D3D3", "#D3D3D3", "#D3D3D3", "#D3D3D3", "#D3D3D3", "#72BCD4", "#D3D3D3"]
sns.barplot(
      y="cnt",
      x="weekday",
      data=weekday_df,
      palette=colors3,
      ax=ax[0]
  )

ax[0].set_title("Penggunaan rental sepeda berdasarkan hari", loc="center", fontsize=35)
ax[0].set_ylabel(None)
ax[0].set_xlabel(None)
ax[0].tick_params(axis='y', labelsize=25)
ax[0].tick_params(axis='x', labelsize=20)

colors4 = ["#D3D3D3", "#72BCD4"]
sns.barplot(
      y="cnt",
      x="workingday",
      data=workingday_df,
      palette=colors4,
      ax=ax[1]
  )

ax[1].set_title("Pengelompokkan berdasarkan hari kerja dan hari libur", loc="center", fontsize=35)
ax[1].set_ylabel(None)
ax[1].set_xlabel(None)
ax[1].tick_params(axis='y', labelsize=25)
ax[1].tick_params(axis='x', labelsize=20)
st.pyplot(fig)

### Pertanyaan 3:

In [ ]:
st.subheader("Berdasarkan Waktu")
fig, ax = plt.subplots(figsize=(25, 15))

colors5 = [ "#D3D3D3", "#72BCD4", "#D3D3D3", "#D3D3D3"]
sns.barplot(
    y="cnt",
    x="hour_group",
    data=bytime_df,
    palette=colors5,
)

ax.set_title("Penggunaan rental sepeda berdasarkan rentang waktu", loc="center", fontsize=35)
ax.set_ylabel(None)
ax.set_xlabel(None)
ax.tick_params(axis='y', labelsize=25)
ax.tick_params(axis='x', labelsize=20)
st.pyplot(fig)

## Conclusion